https://mattbaker.blog/2019/01/28/the-stern-brocot-tree-hurwitzs-theorem-and-the-markoff-uniqueness-conjecture/

In [13]:
import itertools as it

class SternBrocotNode:
    root_left_tuple, root_middle_tuple, root_right_tuple = (0, 1), (1, 1), (1, 0)

    @classmethod
    def locate_fraction_tuple(cls, fraction_tuple):
        N = cls([], cls.root_left_tuple, cls.root_middle_tuple, cls.root_right_tuple)
        while True:
            fraction_tuple_value = fraction_tuple[0] / fraction_tuple[1]
            node_middle_tuple_fraction_value = N.middle_tuple[0] / N.middle_tuple[1]
            if fraction_tuple_value < node_middle_tuple_fraction_value:
                N = N.L()
            elif fraction_tuple_value > node_middle_tuple_fraction_value:
                N = N.R()
            else:
                return N
    
    @staticmethod
    def mediant(x, y):
        return (x[0] + y[0], x[1] + y[1])

    def __init__(self, move_list, left_tuple, middle_tuple, right_tuple):
        self.move_list = move_list
        self.left_tuple = left_tuple
        self.middle_tuple = middle_tuple
        self.right_tuple = right_tuple

    def L(self):
        """Left child: (left_tuple, mediant(left_tuple, middle_tuple), middle_tuple)."""
        new_move_list = self.move_list + ['L']
        new_middle_tuple = type(self).mediant(self.left_tuple, self.middle_tuple)
        return type(self)(new_move_list, self.left_tuple, new_middle_tuple, self.middle_tuple)

    def R(self):
        """Right child: (middle_tuple, mediant(middle_tuple, right_tuple), right_tuple)."""
        new_move_list = self.move_list + ['R']
        new_middle_tuple = type(self).mediant(self.middle_tuple, self.right_tuple)
        return type(self)(new_move_list, self.middle_tuple, new_middle_tuple, self.right_tuple)
    
    def run_list(self):
        return [(k, len(list(g))) for k, g in it.groupby(self.move_list)]
    
    def cf_list(self):
        _run_list = self.run_list()
        if _run_list[0][0] == 'L':
            _run_list = [('R', 0)] + _run_list
        cf_list = [run[1] for run in _run_list]
        return cf_list
    
    def __eq__(self, other):
        return self.move_list == other.move_list

    def __repr__(self):
        return f"{type(self).__name__}({self.move_list}, {self.left_tuple}, {self.middle_tuple}, {self.right_tuple})"
    
    def __str__(self):
        return f"{str(self.run_list())}\n{self.middle_tuple}"
    

if __name__ == "__main__":
    fraction_tuple = (23, 196)
    sb_node = SternBrocotNode.locate_fraction_tuple(fraction_tuple)
    print(repr(sb_node))
    print(str(sb_node))
    print(bool(sb_node))

SternBrocotNode(['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'R', 'L', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R'], (21, 179), (23, 196), (2, 17))
[('L', 8), ('R', 1), ('L', 1), ('R', 10)]
(23, 196)
True


In [31]:
[(k, len(list(g))) for k, g in it.groupby([1, 2, 2, 2])]

[(1, 1), (2, 3)]

In [34]:
[(k, len(list(g))) for k, g in it.groupby([1, 2, 2, 2, 1, 1, 2, 1, 2])]

[(1, 1), (2, 3), (1, 2), (2, 1), (1, 1), (2, 1)]